In [ ]:
import nltk
import numpy as np
import pandas as pd

In [ ]:
import random
from sklearn.model_selection import train_test_split
import pprint, time

***Treebank corpus***

In [ ]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

***Universal tagset***

In [ ]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

***Reading the Treebank tagged sentences***

In [ ]:
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [ ]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


***Split data into training and validation set in the ratio 80:20***

In [ ]:
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

***Create list of train and test tagged words***

In [ ]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


***Checking some of the tagged words***

In [ ]:
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

***Unique tags in training data***

In [ ]:
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
#total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'DET', 'ADV', 'PRT', 'X', 'ADP', 'ADJ', '.', 'VERB', 'PRON', 'CONJ', 'NOUN', 'NUM'}


***Emission Probability***

In [ ]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

***Transition Probability***

In [ ]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

***t x t transition matrix of tags***

In [ ]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[6.03708485e-03 1.20741697e-02 2.87480245e-04 4.51343954e-02
  9.91806854e-03 2.06410810e-01 1.73925534e-02 4.02472317e-02
  3.30602261e-03 4.31220367e-04 6.35906279e-01 2.28546783e-02]
 [7.13731572e-02 8.14584941e-02 1.47401085e-02 2.28859577e-02
  1.19472459e-01 1.30721495e-01 1.39255241e-01 3.39022487e-01
  1.20248254e-02 6.98215654e-03 3.21955010e-02 2.98681147e-02]
 [1.01369865e-01 9.39334650e-03 1.17416831e-03 1.21330721e-02
  1.95694715e-02 8.29745606e-02 4.50097844e-02 4.01174158e-01
  1.76125243e-02 2.34833662e-03 2.50489235e-01 5.67514673e-02]
 [5.68902567e-02 2.57543717e-02 1.85085520e-01 7.57255405e-02
  1.42225638e-01 1.76821072e-02 1.60868734e-01 2.06419379e-01
  5.41995019e-02 1.03786280e-02 6.16951771e-02 3.07514891e-03]
 [3.20931405e-01 1.45532778e-02 1.26550242e-03 3.45482156e-02
  1.69577319e-02 1.07061505e-01 3.87243740e-02 8.47886596e-03
  6.96026310e-02 1.01240189e-03 3.23588967e-01 6.32751212e-02]
 [5.24271838e-03 5.24271838e-03 1.14563107e-02 2.09708735e-02
  8

In [ ]:
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,DET,ADV,PRT,X,ADP,ADJ,.,VERB,PRON,CONJ,NOUN,NUM
DET,0.006037,0.012074,0.000287,0.045134,0.009918,0.206411,0.017393,0.040247,0.003306,0.000431,0.635906,0.022855
ADV,0.071373,0.081458,0.014740,0.022886,0.119472,0.130721,0.139255,0.339022,0.012025,0.006982,0.032196,0.029868
PRT,0.101370,0.009393,0.001174,0.012133,0.019569,0.082975,0.045010,0.401174,0.017613,0.002348,0.250489,0.056751
X,0.056890,0.025754,0.185086,0.075726,0.142226,0.017682,0.160869,0.206419,0.054200,0.010379,0.061695,0.003075
ADP,0.320931,0.014553,0.001266,0.034548,0.016958,0.107062,0.038724,0.008479,0.069603,0.001012,0.323589,0.063275
ADJ,0.005243,0.005243,0.011456,0.020971,0.080583,0.063301,0.066019,0.011456,0.000194,0.016893,0.696893,0.021748
.,0.172192,0.052569,0.002789,0.025641,0.092908,0.046132,0.092372,0.089690,0.068769,0.060079,0.218539,0.078210
VERB,0.133610,0.083886,0.030663,0.215930,0.092357,0.066390,0.034807,0.167956,0.035543,0.005433,0.110589,0.022836
PRON,0.009567,0.036902,0.014123,0.088383,0.022323,0.070615,0.041913,0.484738,0.006834,0.005011,0.212756,0.006834
CONJ,0.123491,0.057080,0.004391,0.009330,0.055982,0.113611,0.035126,0.150384,0.060373,0.000549,0.349067,0.040615


# ***Viterbi Algorithm***

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

***Testing Viterbi algorithm***

In [ ]:
rndom = [random.randint(1,len(test_set)) for x in range(10)]

In [ ]:
test_run = [test_set[i] for i in rndom]

In [ ]:
test_run_base = [tup for sent in test_run for tup in sent]

In [ ]:
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_tagged_words

['In',
 'a',
 'post-hearing',
 'news',
 'conference',
 ',',
 'Mr.',
 'Phelan',
 ',',
 'who',
 '*T*-214',
 'has',
 'publicly',
 'expressed',
 'concern',
 'about',
 'market',
 'volatility',
 ',',
 'said',
 '0',
 'he',
 'told',
 'the',
 'House',
 'finance',
 'and',
 'telecommunications',
 'subcommittee',
 'that',
 'he',
 'would',
 'support',
 'the',
 'program-trading',
 'halt',
 'proposal',
 '``',
 '*',
 'providing',
 '0',
 'the',
 'SEC',
 'would',
 'be',
 'comfortable',
 'with',
 'the',
 'language',
 "''",
 'in',
 'a',
 'bill',
 '.',
 'Later',
 'yesterday',
 ',',
 'a',
 'Massachusetts',
 'senate',
 'committee',
 'approved',
 'a',
 'bill',
 '0',
 '*T*-1',
 'to',
 'allow',
 'national',
 'interstate',
 'banking',
 'by',
 'banks',
 'in',
 'the',
 'state',
 '*',
 'beginning',
 'in',
 '1991',
 '.',
 'But',
 'a',
 'takeover',
 'battle',
 'opens',
 'up',
 'the',
 'possibility',
 'of',
 'a',
 'bidding',
 'war',
 ',',
 'with',
 'all',
 '0',
 'that',
 'implies',
 '*T*-1',
 '.',
 'Douglas',
 'Madiso

In [ ]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  69.76910638809204
Viterbi Algorithm Accuracy:  92.3529411764706


In [ ]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

# ***Modified Viterbi Algorithm***

In [ ]:
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
#test accuracy on subset of test data 
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  68.83240032196045
Viterbi Algorithm Accuracy:  97.05882352941177


In [ ]:
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)
#Will and Marry are tagged as NUM as they are unknown words for Viterbi Algorithm

[('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('Will', 'DET'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'DET')]
